In [2]:
import networkx as nx
import math

In [3]:
#读取
G1 = nx.read_edgelist('seed_G1.edgelist')
G2 = nx.read_edgelist('seed_G2.edgelist')
with open('seed_node_pairs.txt','r') as sp:
    lines = sp.readlines()

In [4]:
#获得G1 匹配的  与 G2 匹配的 并存入字典
g1p = []
g1pdir = {}
g2p =[]

In [5]:
for line in lines:
    line = line.strip('\n')
    line = line.split(' ')
    g1p.append(line[0])
    g2p.append(line[1])
    g1pdir[line[0]] = line[1]

In [6]:

l = []
for i in G1.nodes:
    l.append(int(i))
y = sorted(l)
with open('G1.txt','w') as f:
    for j in range(len(y)):
        f.write(str(y[j])+ "\n") 


In [11]:

#获得未匹配过的 G1 与 G2 的 点
g1np = []
g2np = []
for i in G1.nodes:
    if i not in g1p:
        g1np.append(i)
for j in G2.nodes:
    if j not in g2p:
        g2np.append(j)

In [13]:
print(len(g1p))
print(len(g1np))
print(nx.info(G1))

500
4063
Name: 
Type: Graph
Number of nodes: 4563
Number of edges: 159748
Average degree:  70.0188
